
# Tic Tac Toe Reinforcement Learning Agent

Dieses Notebook implementiert einen Reinforcement Learning (RL)-Algorithmus, der Q-Learning verwendet, damit der Computer lernt, wie man Tic Tac Toe spielt.

Das Spielbrett ist für ein 3x3-Gitter ausgelegt, und der Computer lernt, durch "Exploration" und "Exploitation" optimale Züge zu machen und seine Strategie auf der Grundlage des Q-Learning-Ansatzes zu verbessern.

---




Ihre Umgebung sollte mithilfe der Informationen aus dem README.md vorbereitet sein und Sie können nun mit dem Notebook fortfahren.


### Step 1: Spielumgebung 

In diesem Abschnitt instanzieren wir ein Game Objekt vom Typ "Game", welches im Game.py Dokument als Klasse definiert wurde.

Die Methoden in der Klasse sollten Ihnen bereits bekannt vorkommen. Es sind die selben wie beim Tic Tac Toe Spiel, welches Sie bereits kennengelernt haben.


In [ ]:
from Game import Game

myGame = Game()


### Step 2: Q-Learning Algorithm

In diesem Abschnitt instanzieren wir den Q-Learning-Agenten. Die Klasse des Agenten wird im QLearningAgent.py Dokument definiert.
Die Klasse enthält Methoden, die es dem Agenten erlauben mit dem Spiel zu spielen. Ausserdem besitzt die Klasse eine Methode train_agent() damit der Agent aus Erfahrungen lernen kann. Wir verwenden eine Q-Tabelle, um die Werte der Aktionen für jeden Spielzustand zu speichern.

- myGame: Die Umgebung oder das Spiel, mit dem der Agent interagiert.
- 0.5: Die Lernrate (Alpha) - Bestimmt, wie stark neue Informationen die gelernten Werte beeinflussen.
- Ein Wert von 0,5 bedeutet, dass der Agent neue Informationen und frühere Erfahrungen gleich gewichtet.
- 0.9: Der Diskontierungsfaktor (Gamma) - Bestimmt die Bedeutung zukünftiger Belohnungen im Vergleich zu sofortigen Belohnungen.
- Ein Wert nahe 1,0 bedeutet, dass der Agent langfristigen Belohnungen den Vorzug gibt.
- 0.1: Die Erkundungsrate (epsilon) - Die Wahrscheinlichkeit, dass der Agent erkundet (eine zufällige Aktion wählt)
- statt bekannte Informationen zu nutzen. Ein niedriger Wert begünstigt die Ausbeutung gegenüber der Erkundung.

In [ ]:
from QLearningAgent import QLearningAgent

agent = QLearningAgent(myGame, 0.5, 0.9, 0.1)


### Step 3: Training des Agenten

In diesem Abschnitt trainieren wir den Agenten, indem wir ihn mehrere Episoden lang gegen einen Gegner spielen lassen, welcher zufällige Züge ausführt.


In [ ]:
EPISODES = 100000
bot_wins, opponent_wins, draws = agent.train_agent(EPISODES)


### Step 4: Inspizieren des trainierten RL-Agenten

Nachdem der Reinforcement-Learning-Agent trainiert wurde, ist es entscheidend, sein Verhalten zu analysieren und zu verstehen, was er tatsächlich gelernt hat. Nur weil der Agent im Training gute Ergebnisse erzielt hat, bedeutet das nicht automatisch, dass er auch robuste oder nachvollziehbare Strategien entwickelt hat. Durch eine gezielte Inspektion können wir überprüfen, ob das Lernverhalten sinnvoll ist, ob sich Muster erkennen lassen und ob es möglicherweise noch unerwartete Schwächen gibt. Diese Analyse ist ein wichtiger Schritt, um die Qualität und Korrektheit des Trainingsprozesses zu validieren.

Zunächst überprüfen wir die Anzahl der nicht-nullen Q-Werte in der Q-Tabelle im Vergleich zur Gesamtzahl aller möglichen Einträge. Dies gibt uns einen Eindruck davon, wie viele Zustände überhaupt während des Trainings relevant wurden und für welche der Agent tatsächlich Entscheidungswerte gelernt hat.

In [ ]:
import numpy as np
from collections import defaultdict

q_table = defaultdict(lambda: np.zeros(9), agent.q_table)

non_zero_entries = sum(np.any(values) for values in q_table.values())
non_zero_entries, len(q_table)  # Nicht-Null-Einträge gegenüber Gesamteinträgen in der Q-Tabelle




Anschliessend visualisieren wir die Lernstatistik in Form eines Diagramms. Hierbei wird die Siegquote über den Trainingsverlauf hinweg dargestellt, um die Lernkurve des Agenten nachvollziehen zu können.

In [ ]:
import matplotlib.pyplot as plt

# Die Lernstatistik als Grafen darstellen
plt.plot(range(EPISODES), bot_wins, label="Bot gewinnt", color="blue")
plt.plot(range(EPISODES), opponent_wins, label="Gegner gewinnt", color="red")
plt.plot(range(EPISODES), draws, label="Unentschieden", color="green")
plt.xlabel("Episoden")
plt.ylabel("Anz. Siege/Unentschieden")
plt.title("Spielstatistik während der Lernphase")
plt.legend()
plt.show()



Für eine detaillierte Betrachtung geben wir ausserdem für alle Zustände, in denen der Agent sinnvolle Werte gelernt hat (z. B. wenn höchstens XX (kann selber definiert werden) der 9 möglichen Aktionen einen Q-Wert von 0 haben), die Q-Werte als 3×3-Matrix aus. Dies ermöglicht eine intuitive Interpretation, welche Aktionen in welcher Spielsituation bevorzugt werden.

In [ ]:
import numpy as np

min_empty_fields = 6

for idx, values in q_table.items():
    # es wird für jeden Key (Spielzustand) gezählt, wie viele freie Felder im Value (alle Q-Werte) enthalten sind
    space_count = sum(row.count(' ') for row in idx)
    if space_count >= min_empty_fields and np.any(values < 0):  # Bedingung um herauszufiltern, wo mindestens XX Felder im State noch leer sind
        # Spielzustand als 3x3-Matrix ausgeben
        print("Board State:")
        for row in idx:
            print(row)

        # Q-Werte als 3x3-Matrix ausgeben
        print("\nQ-Values:")
        for i in range(0, 9, 3):
            print(values[i:i + 3])

        print("\n" + "-" * 20 + "\n")  # Trennlinie zur besseren Lesbarkeit



Zum Schluss erstellen wir für ausgewählte konkrete Zustände sogenannte Heatmaps. Diese zeigen visuell auf einen Blick, wie stark der Agent jede mögliche Aktion in einem bestimmten Zustand bewertet. Sie helfen dabei, das Entscheidungsverhalten des Agenten besser zu verstehen und mit strategisch sinnvollen Spielzügen im TicTacToe zu vergleichen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_q_values(state):
    """Visualizes Q-values for a specific Tic Tac Toe game state."""
    state = tuple(tuple(row) for row in state)  # State in ein Tupel von Strings übersetzen: entspricht dem key im Q-table
    q_values = q_table.get(state, np.zeros(9))  # Q-Werte für den State abrufen, falls key nicht existiert alle Q-Werte mit zero besetzen
    
    # Shape der Q-Werte an ein 3x3 board layout anpassen
    q_values_board = q_values.reshape((3, 3))

    # Heatmap plotten
    plt.figure(figsize=(6, 5))
    sns.heatmap(q_values_board, annot=True, cmap="coolwarm", cbar=True, square=True)
    plt.title("Q-values Heatmap for Selected Game State")
    plt.xlabel("Column")
    plt.ylabel("Row")
    plt.show()

# Beispielhafter Zustand zur Veranschaulichung
example_state = [['X', ' ', 'X'], 
                ['O', 'O', ' '], 
                [' ', 'X', ' ']]
visualize_q_values(example_state)



### Step 5: Exportieren des trainierten RL-Agenten

Nach dem erfolgreichen Training möchten wir den gelernten Wissensstand des RL-Agenten sichern, um ihn später wiederverwenden oder analysieren zu können – ohne erneut trainieren zu müssen. 

Zu diesem Zweck speichern wir die Q-Tabelle, die die erlernten Zustands-Aktions-Werte enthält, in einer externen Datei. Dies ermöglicht es uns, den Agenten zu einem späteren Zeitpunkt zu laden und direkt in einem Spiel oder in einer Evaluierungsumgebung einzusetzen.

Typischerweise erfolgt der Export im Format als Pickle-Datei (`.pkl`): Damit kann die Q-Tabelle im ursprünglichen Python-Format gespeichert und wieder eingelesen werden.

Vorteile des Exports:
- Keine erneute Trainingszeit notwendig
- Ermöglicht Analyse, Visualisierung oder Vergleich mehrerer Trainingsläufe
- Grundlage für den Einsatz des Agenten im "echten Spielbetrieb"

In [ ]:
import pickle

with open("trained_q_table.pkl", "wb") as f:
    pickle.dump(dict(agent.q_table), f) 
print("Model saved as 'trained_q_table.pkl'")


🔍 Wurde die Exportdatei erfolgreich erstellt?

Bitte überprüfe, ob im Projektverzeichnis eine neue Datei (z. B. trained_q_table.pkl) erstellt wurde. Wenn die Datei vorhanden ist, war der Export erfolgreich und der trainierte Agent kann später wieder geladen werden.

In der nächsten Zelle wird noch gezeigt, wie man die Werte wieder laden und einem Agenten zuweisen kann.

In [ ]:
# Die Pickle-Datei wird eingelesen
with open("trained_q_table.pkl", "rb") as f:
    q_table_data = pickle.load(f)
    # Sicherstellen, dass q_table nach dem Laden vom Typ defaultdict ist
    agent.q_table = defaultdict(lambda: np.zeros(9), q_table_data)